In [109]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
import pylab as pl
import matplotlib.pyplot as plt
%matplotlib inline
import scipy as sp
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [110]:
data = pd.read_excel("C:\Users\Niku\Documents\ML Project\crimetest.xlsx")
print data.shape

(10, 56)


In [111]:
X = data[['medIncome', 'householdsize']]
y = data[['Category']]
X = np.matrix(X)
y = np.matrix(y)

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state=1)
print X_train.shape
print X_test.shape
print y_test.shape
print type(y_train)

(5L, 2L)
(5L, 2L)
(5L, 1L)
<class 'numpy.matrixlib.defmatrix.matrix'>


In [113]:
def g(z):
    temp = 1 + np.exp(-z)
    temp2 = 1/temp
    return temp2

In [114]:
def costFunction(theta, X, y, alpha):
    theta = np.matrix(theta)
    first = np.multiply(-y, np.log(g(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - g(X * theta.T)))
    reg = (alpha / 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / (len(X)) + reg
    return cost

In [115]:
def gradient(theta, X, y, alpha):
    X = np.matrix(X)
    y = np.matrix(y)
    theta = np.matrix(theta)
    hofx = g(X*theta.T)
    err = hofx - y
    term1 = ((X.T * err) / len(X)).T 
    term2 = ((alpha / len(X)) * theta)
    grad = term1 + term2
    zero_intercept = np.multiply(err, X[:,0])
    grad[0, 0] = np.sum(zero_intercept)
    return np.array(grad).ravel()

In [116]:
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, learning_rate):
    rows = X.shape[0]
    params = X.shape[1]
    
    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))
    
    # insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))
        
        # minimize the objective function
        fmin = minimize(fun=costFunction, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x
    
    return all_theta

In [117]:
X = np.matrix(X)
y = np.matrix(y)
rows = X.shape[0]
params = X.shape[1]
all_theta = np.zeros((1, params + 1))

X = np.insert(X, 0, values=np.ones(rows), axis=1)

theta = np.zeros(params + 1)

y_0 = np.array([1 if label == 0 else 0 for label in y])
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, all_theta.shape

((10L, 3L), (10L, 1L), (3L,), (1L, 3L))

In [118]:
all_theta

array([[ 0.,  0.,  0.]])

In [119]:
theta = np.array(theta)
all_theta = one_vs_all(theta, X, y, 1, 0.1)
all_theta.shape

TypeError: one_vs_all() takes exactly 4 arguments (5 given)

In [ ]:
theta = np.array(theta)

In [ ]:
y_pred = predict_all(X_test, all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y_test)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print 'accuracy = {0}%'.format(accuracy * 100)


In [ ]:
def one_vs_all(theta, X, y, num_labels, learning_rate):
    X = np.matrix(X)
    y = np.matrix(y)
    theta = np.array(theta)
    rows = X.shape[0]
    params = X.shape[1]
    y = np.matrix(y)
    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))
    
    # insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))
        
        # minimize the objective function
        fmin = minimize(fun=costFunction, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x
    
    return all_theta